<img src='https://www.anadronestarting.com/wp-content/uploads/intel-main_opt.png' width=50%>

# 모바일넷을 이용한 이미지분류_파일 변환
### (keras .h5 to tensorflow .pb to OpenVINO .xml) 
<font size=5><b>(Image Classification using Mobilenet and OpenVINO)<b></font>

<div align='right'>성  민  석 (Minsuk Sung)</div>
<div align='right'>이  인  구 (Ike Lee)</div>

<img src='https://gitcdn.xyz/cdn/Tony607/blog_statics/9ddd48eb623175fb734e63865ebe604351c64c61/images/openvino/openvino.png' width=50%>

---

<h1>강의목차<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Keras-h5-파일을-Tensorflow-pb파일로-변환" data-toc-modified-id="Keras-h5-파일을-Tensorflow-pb파일로-변환-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Keras h5 파일을 Tensorflow pb파일로 변환</a></span></li><li><span><a href="#Tensorflow-pb파일을-OpenVINO-xml,-bin파일로-변환" data-toc-modified-id="Tensorflow-pb파일을-OpenVINO-xml,-bin파일로-변환-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Tensorflow pb파일을 OpenVINO xml, bin파일로 변환</a></span></li><li><span><a href="#OpenVINO에서-xml,bin파일로-추론" data-toc-modified-id="OpenVINO에서-xml,bin파일로-추론-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>OpenVINO에서 xml,bin파일로 추론</a></span></li><li><span><a href="#Reference" data-toc-modified-id="Reference-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Reference</a></span></li></ul></div>

<img src='https://gitcdn.xyz/cdn/Tony607/blog_statics/73e93019d8a9dafd0b5a0fe200e982b75d50bf21/images/keras-tf/keras_tf_pb.png' width=50%>

## Keras h5 파일을 Tensorflow pb파일로 변환

In [ ]:
%run ./utils/keras_to_freeze_binary.py ./bin/mobilenetv2_class20.h5 ./bin/mobilenetv2

<img src='https://software.intel.com/sites/default/files/managed/b7/e3/webops12032-diagram-of-flow-750.jpg'>

## Tensorflow pb파일을 OpenVINO xml, bin파일로 변환

In [ ]:
%run /opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --framework tf \
    --input_model ./bin/mobilenetv2.pb \
    --batch 1 \
    --data_type FP32 \
    --output_dir bin/

## OpenVINO에서 xml,bin파일로 추론

In [ ]:
import os
# assert 'computer_vision_sdk' in os.environ['PYTHONPATH']

from PIL import Image
import numpy as np

try:
    from openvino import inference_engine as ie
    from openvino.inference_engine import IENetwork, IEPlugin
except Exception as e:
    exception_type = type(e).__name__
    print("The following error happened while importing Python API module:\n[ {} ] {}".format(exception_type, e))
    sys.exit(1)
    
    
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
def pre_process_image(imagePath, img_height=224):
    # Model input format
    n, c, h, w = [1, 3, img_height, img_height]
    image = Image.open(imagePath)
    processedImg = image.resize((h, w), resample=Image.BILINEAR)

    # Normalize to keep data between 0 - 1
    processedImg = (np.array(processedImg) - 0) / 255.0

    # Change data layout from HWC to CHW
    processedImg = processedImg.transpose((2, 0, 1))
    processedImg = processedImg.reshape((n, c, h, w))

    return image, processedImg, imagePath

In [ ]:
# Plugin initialization for specified device and load extensions library if specified.
plugin_dir = None
model_xml = './bin/mobilenetv2.xml'
model_bin = './bin/mobilenetv2.bin'

In [ ]:
# Devices: GPU (intel), CPU, MYRIAD
# plugin = IEPlugin("MYRIAD", plugin_dirs=plugi)
plugin = IEPlugin("CPU")

In [ ]:
# Read IRn_dir
net = IENetwork.from_ir(model=model_xml, weights=model_bin)
assert len(net.inputs.keys()) == 1
assert len(net.outputs) == 1
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))

In [ ]:
# Load network to the plugin
exec_net = plugin.load(network=net)
del net

In [ ]:
import os
test_list = list(map(lambda x:x.split('.')[0],os.listdir('./img/test/')))
print(test_list)

In [ ]:
# Run inference
input_file = input('Select Test File : ')

fileName = './img/test/{}.jpg'.format(input_file)

In [ ]:
image, processedImg, imagePath = pre_process_image(fileName)
res = exec_net.infer(inputs={input_blob: processedImg})
# Access the results and get the index of the highest confidence score
output_node_name = list(res.keys())[0]
res = res[output_node_name]

In [ ]:
# Predicted class index.
idx = np.argsort(res[0])[-1]

In [ ]:
# decode the predictions
with open('./bin/class20.pickle', 'rb') as f:
    labels = pickle.load(f)
    
plt.imshow(load_img(fileName))
plt.show()    
print('Predicted : {} ({:.2f}%)'.format(labels[idx].upper(),res[0][idx]*100))



In [ ]:
import pandas as pd
pd.DataFrame(res,columns=labels.values()).T.plot(kind='bar')
plt.show()

## Reference

- Intel OpenVINO
    - https://software.intel.com/en-us/openvino-toolkit

